# <center><font face="arial" size="5" color=#0101DF>NUMERIQUE ET SCIENCES INFORMATIQUES 1ère NSI</font></center>

## <font color=#013ADF>Séquence N° 3 : projet STEGANOGRAPHIE</font>

L'ensemble du projet doit être réalisé en 5 heures classe. Le dossier informatisé est à rendre à l'issue de ces séances. Le fond et la forme du dossier est la même que lors du projet précédent.

L'organisation et la notation est également la même. Seuls les rôles changent !
<div class="alert alert-danger" role="alert">
<img src="Images/warning.png" alt="Attention" width=5% align=left>

**Le travail individuel hors classe est interdit, à l'exception de la rédaction du dossier. Par contre, il n'est pas interdit aux développeurs de se former ;-)**

**Problématique** :

La répression sévit en République Démocratique du KIRPUSTI. La censure fait rage, peu d'informations sortent du pays. Vous décidez avec votre ami kirpustian Gaspieux d'utiliser la technique de stéganographie.

**Programme de décodage (lecture du message dans l'image)**

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-98 -*-
"""
@author: Gaspieux

Stéganographie : on cache une message se terminant par le caractère #
dans une image.
Le principe est le suivant : Comme l'oeil humain est peu sensible à une légère
nuance de couleur, on modifie éventuellement un bit de couleurs pour coder
le message.
Pour coder un 0, la composante de couleur qui le code doit avoir une valeur paire
Pour coder un 1, la composante de couleur qui la code doit avoir une valeur impaire
On vient ajuster la valeur si besoin.
Ici le programme de lecture du message dans l'image
"""

# importation de librairie
import sys
from PIL import Image


def ecriture_fichier(nom_fichier:str,message:str)->None:
    """
    Procédure d'écriture du message
    sur le disque, dans le répertoire courant
    """
    with open(nom_fichier, 'w', encoding="utf-8") as fichier_texte:
        fichier_texte.write(message)

def recup_couleurs_image(image_file)->list:
    """
    Cette fonction prend en entrée le nom de l'image 
    et renvoie une liste qui contient la valeur des 
    composantes rouge, vert et bleu de chaque pixel
    de l'image en décimal.
    """
    try:
        nom_image= Image.open(image_file)
    except :
        print ('Erreur sur ouverture du fichier ',image_file)
        sys.exit(1)
    largeur,hauteur=nom_image.size
    # balayage de l'image colonne par colonne de gauche à droite
    liste=[]
    for i in range(0,largeur):
        for j in range(0,hauteur):
            r,g,b = nom_image.getpixel((i,j))
            liste.append(r)
            liste.append(g)
            liste.append(b)
    nom_image.close()
    return liste

def decode_message(couleurs:list)->str:
    """
    lire les pixels de l'image si pair 0 sinon impair 1
    et renvoie le texte sans le caractère final #
    """
    text=""
    chaine=""
    cpt=0
    for i in couleurs:
        if i%2==0:
            chaine+='0'
        else:
            chaine+='1'
        cpt+=1
        if cpt==8:
            cpt=0
            nb_dec=0
            for power, digit in enumerate(chaine[::-1]):
                nb_dec+=int(digit)*(2**power)
            caractere=chr(nb_dec)
            if caractere=='#':
                return text
            text+=caractere
            chaine=""

#ouverture du fichier image
fichier_image = input('Indiquer le nom de l\'image dans laquelle le message est caché (nom + extension) : ')
fichier_texte=input('Indiquer le nom du fichier dans lequel écrire le message (nom + extension) : ')
# création d'une liste des couleurs des pixels de l'image
couleurs=recup_couleurs_image(fichier_image)
message_decode=decode_message(couleurs)
ecriture_fichier(fichier_texte,message_decode)
print("Traitement terminé !")

KeyboardInterrupt: Interrupted by user

Un algorithme " gros grain" du programme ci-dessus serait de la forme :

- Ouvrir le fichier image qui contient le message ;

- Lire les composantes de couleurs des pixels de l'image ;

- En déduire la succession des bits (0 ou 1) ;

- Regrouper les bits par octet pour retrouver le caractère codé ;

- Arrêter le traitement si le caractère reconnu est "#" ;

- Écrire le message dans un fichier texte.

<div class="alert alert-warning">
<img src="Images/CR.png" alt="logo CR" width=5% align=right>

On vous demande d'étudier le programme pour comprendre comment le message est reconstitué. On détaillera plus particulièrement la fonction "decode_message" en traçant son algorithme. (**voir remarque en bas du document**)

**Voilà ce qui a pu être sauvé du programme de codage.**

In [ ]:
#Programme principal
fichier_image = input('Indiquer le nom de l\'image dans laquelle on cache le message (nom + extension) : ')
fichier_texte=input('Indiquer le nom du fichier dans lequel lire le message (nom + extension) : ')

# Récupère les caractéristiques de l'image originale
carac_image=infos_image(fichier_image)
# Création du nom de fichier de sauvegarde
fichier_sauvegarde=creer_nom_sauvegarde(fichier_image, carac_image)
# création d'une liste des couleurs des pixels de l'image
couleurs=recup_couleurs_image(fichier_image)
# Texte à cacher dans l'image. le caractère # indique que le message est terminé
# Ce message est dans un fichier texte appelé message.txt
texte=lecture_fichier(fichier_texte)
# création d'un chaine de bits à écrire sur l'image
chaine_binaire=conv_message_vers_binaire(texte)

if len(chaine_binaire)>len(couleurs):
    print("Le message est trop long pour l'image utilisée")
else:
    couleurs_mod=encode_message(chaine_binaire,couleurs)
    creation_image(carac_image, couleurs_mod, fichier_sauvegarde)
    print("Traitement terminé !")

À la lecture du programme partiel de codage et d'après ce que nous connaissons de l'algorithme de décodage, nous pouvons déduire qu'il faut :

Écrire le message à cacher dans un fichier texte avec un éditeur de texte. Finir son écriture par le caractère # ;

Nous voyons également des appels à des fonctions qui d'après leurs noms, devraient réaliser les tâches précitées.

In [10]:
def infos_image(image_file)->tuple:
    """
    Renvoie les dimensions, le format (e.g. PNG)
    et le mode (e.g. RGB) de l'image
    originale si elle est trouvée !
    """
    try:
        mon_image = Image.open(image_file)
        largeur, hauteur = mon_image.size
        return largeur,hauteur,mon_image.format,mon_image.size, mon_image.mode
    except:
        print("Erreur : Fichier non trouvé")
    pass


if __name__=='__main__':
    print(infos_image('aaa.png'))

Erreur : Fichier non trouvé


In [ ]:
def recup_couleurs_image(image_file)->list:
    """
    Cette fonction prend en entrée le nom de l'image et
    renvoie une liste qui contient la valeur des composantes
    rouge, verte et bleue de chaque pixel de l'image en décimal.
    """
    infos_image('aa.png')
    i=0
    for x in range(largeur):
        for y in range(hauteur):
            (rouge,vert,bleu) = mon_image.getpixel((x,y)) 
            T[i][0]=(x;y) 
            T[i][1]= (Rouge, vert, bleu) 
    pass


In [8]:
def lecture_fichier(nom_fichier)->str:
    """
    Fonction réalisant la lecture du fichier
    texte à écrire sur une image
    """
    with open(nom_fichier, 'r', encoding="utf-8") as fichier_texte:
        lignes_texte=fichier_texte.readlines()
    return lignes_texte
    pass

In [7]:
lecture_fichier("texte.txt")

In [ ]:
def conv_message_vers_binaire(texte:str)->str:
    """
    Cette fonction prend en entrée, le texte qui doit être caché dans l'image
    et renvoie une chaine de caractère qui contient le code binaire représentant
    chaque caractère du message à cacher sur 8 bits.
    """
    pass

In [ ]:
def encode_message(msg:str, coul:list)->list:
    """
    Cette fonction modifie les valeurs (couleurs) de
    la liste passée en paramètre selon le procédé suivant :
    si le nombre représentant la couleur est pair et le code binaire
    à écrire est 1, on doit rendre le nombre représentant la couleur impair.
    De même si le code binaire à écrire est 0 et que le nombre représentant
    la couleur est impair, on doit rendre le nombre représentant la couleur pair.
    """
    pass

In [ ]:
def creation_image(caract:tuple, coul:list, image_file)->None:
    """
    Cette procédure crée une image ayant les mêmes
    caractéristiques que l'orignal mais avec les
    nouvelles couleurs, puis la sauvegarde avec
    un nom passé en argument
    """
    pass

In [ ]:
def creer_nom_sauvegarde(nom:str, caract:tuple)->str:
    """
    Fonction qui génère un nom pour la future image
    à sauvegarder, en relation avec le nom initial
    """
    pass

<div class="alert alert-danger" role="alert">
<img src="Images/warning.png" alt="Attention" width=5% align=left>

Vous devez élaborer le document de vision, les users-stories, le document de recette pour réaliser votre application en mode projet et élaborer votre dossier.

Remarques :
- Dans un soucis d'organisation, les développeurs peuvent tracer l'algorithme du programme fourni pendant que le product owner et le scrum master organise les users-stories.
- Il est fortement recommandé de s'inspirer et de réutiliser les documents élaborés lors du projet précédent.
- Les fonctions doivent comporter lorsque c'est possible un test sur les pré-conditions, sur les post-conditions, des test unitaires et des annotations de typage.
